In [1]:
import numpy as np

In [2]:
filePath = "data/ml-100k/u1.base"
with open(filePath, "rt") as dataPath:
    raw_data = dataPath.read().splitlines()
datapoints = [[int(i) for i in data.split("\t")] for data in raw_data]
user_ids = set([datapoint[0] for datapoint in datapoints])
items_ids = set([datapoint[1] for datapoint in datapoints])

In [3]:
user_ratings = {user_id: {} for user_id in user_ids}
for user_id, item_id, rating, timestamp in datapoints:
    user_ratings[user_id][item_id] = (rating, timestamp)

In [4]:
item_ratings = {item_id: {} for item_id in items_ids}
for user_id, item_id, rating, timestamp in datapoints:
    item_ratings[item_id][user_id] = (rating, timestamp)

In [5]:
def precision(var, v_j, var_u, k):
    first = 1 / var * np.dot(v_j, v_j.T)
    second = 1 / var_u * np.eye(k)
    if v_j.shape[1] > 0:
        return first * second
    return second

def eta(r_ij, v_j, k):
    return np.sum(np.array(v_j) * np.array(r_ij), axis=1)
    
def mu(var, prec, et):
    if np.array_equal(prec, np.zeros(prec.shape)):
        return np.zeros(prec.shape[0])
    return np.dot(1 / var * np.linalg.inv(prec), et)
    
def SampleUi(r_ij, v_j, var_u, var, k):
    prec = precision(var, v_j, var_u, k)
    et = eta(r_ij, v_j, k)
    m = mu(var, prec, et)
    return np.random.multivariate_normal(m, prec)

In [6]:
def softmax(numpy_arr):
    s = np.sum([np.exp(i) for i in numpy_arr])
    return np.array([np.exp(i) / s for i in numpy_arr]).reshape(-1)

In [7]:
def stratified_resample(weights):
    N = len(weights)
    # make N subdivisions, and chose a random position within each one
    positions = (np.random.random(N) + range(N)) / N

    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return indexes


def systematic_resample(weights):
    N = len(weights)

    # make N subdivisions, and choose positions with a consistent random offset
    positions = (np.random.random_sample() + np.arange(N)) / N

    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return indexes

In [8]:
from scipy.stats import invgamma

def sample_var(matrix, alpha, beta, k, n_users):
    alpha = n_users * k / 2.0 + alpha
    beta = 0.5 * np.linalg.norm(matrix, 'fro') + beta
    # Generate random value from inverse gamma(shape,scale)
    var = invgamma.rvs(alpha, scale=beta)
    return var

In [10]:
var = 1.0
var_u = 1.0
var_v = 1.0
k = 3
n_particles = 5
alpha = 2
beta = 0.5

particles = [{"u": np.random.normal(size=(len(user_ids), k)),
              "v": np.random.normal(size=(k, len(items_ids))),
              "var_u": 1.0,
              "var_v": 1.0,
              "w": 1 / n_particles} for particle in range(n_particles)]

# user_ratings_seen = {}
users = {}
items = {}

for _idx, (user_id, item_id, rating, timestamp) in enumerate(datapoints):
    
    try:
        user_data = users[user_id]
    except:
        users[user_id] = {"movies_seen": [], "rewards": []}
        user_data = users[user_id]
        
    particle_ind = np.random.choice(range(n_particles), p=[x["w"] for x in particles])
    
    v_tilde = particles[particle_ind]["v"]
    
    v_j = v_tilde[:, user_data["movies_seen"]]
    
    r_ij = user_data["rewards"]
    
    u_i = SampleUi(r_ij, v_j, particles[particle_ind]["var_u"], var, k)
    
    try:
        j = np.argmax(np.dot(u_i, v_j))
    except:
        # if argmax fails, it's because u_i and v_j hav no data yet, just suggest something at random
        j = np.random.randint(len(items_ids))
    
    
    if j in user_ratings[user_id] and user_ratings[user_id][j][0] > 2.5:
        reward = 1
    else:
        reward = 0
        
    try:
        items[j]["rewards"].append(reward)
        items[j]["users_seen"].append(user_id)
    except:
        items[j] = {"rewards": [reward], "users_seen": [user_id]}
        
    users[user_id]["rewards"].append(reward)
                              
    print("reward", reward)
    
    # update posterior
    lambdas = []
    for particle in particles:
        one_over_sig_sqr = np.eye(k) * 1 / particle["var_u"]
        if len(user_data["movies_seen"]) == 0:
            lambdas.append(one_over_sig_sqr)
        else:
            v_js = particle["v"][:, user_data["movies_seen"]]
            lambdas.append(np.dot(v_js, v_js.T) * 1 / var)
    print(particles[0])
    etas = [eta(r_ij, particle["v"][:, user_data["movies_seen"]], k) for particle in particles]
    
    weights = []
    for idx, particle in enumerate(particles):
        try:
            loc = np.dot(particle["v"][:, j], mu(var, lambdas[idx], etas[idx]))
        except:
            loc = np.zeros(1)
        scale = 1 / var * np.dot(np.dot(particle["v"][:, j].T, lambdas[idx]), particle["v"][:, j])
        weights.append(np.random.normal(loc, scale))
    
    weights = softmax(weights)
    
    particle_indices = systematic_resample(weights)
    
    new_particles = []
    for ind in particle_indices:
        particle = particles[ind]
        new_particles.append({
            "u": particle["u"],
            "v": particle["v"],
            "var_u": particle["var_u"],
            "var_v": particle["var_v"],
            "w": 1 / n_particles,
            "lambda": lambdas[ind],
            "eta": etas[ind]
        })
    particles = new_particles
    
    for particle in particles:
        lambda_upd = 1 / var * np.dot(v_tilde[:, j].reshape(-1, 1), v_tilde[:, j].reshape(1, -1))
        particle["lambda"] += lambda_upd
        eta_update = reward * v_tilde[:, j]
        particle["eta"] += eta_update
        u_i = SampleUi(users[user_id]["rewards"],
                       particle["v"][:, user_data["movies_seen"]],
                       particle["var_u"],
                       var,
                       k)
        particle["u"][user_id, :] = u_i
        u_ij = particle["u"][items[j]["users_seen"], :]
        prec_v = np.dot(u_ij, u_ij.T) + 1 / particle["var_v"] * np.eye(k)
        et = np.dot(u_ij.T, np.array(items[j]["rewards"]))
        v_j = np.random.multivariate_normal(var * np.dot(np.linalg.inv(prec_v), et), np.linalg.inv(prec_v))
        particle["v"][:, j] = v_j
        particle["var_u"] = sample_var(particle["u"], alpha, beta, k, len(user_ids))

reward 1
{'u': array([[ 1.33526484,  2.65743739, -2.69150186],
       [-0.00624233,  1.96009068, -0.45382987],
       [ 0.0921997 , -0.13158716, -1.43966898],
       ...,
       [ 0.24631579,  0.25098418, -0.45805947],
       [ 0.13458587,  0.77856621,  1.69913991],
       [ 0.71498672, -0.25923724, -1.42821072]]), 'v': array([[ 0.15685277,  0.33539714,  1.43216316, ..., -0.76516616,
         0.8495931 ,  0.1209573 ],
       [-0.84313629,  0.30588213, -0.20951936, ..., -0.05590119,
         0.16443224,  0.23055965],
       [-0.17931218,  1.02865594, -1.77344224, ..., -0.29146862,
        -1.71199336, -0.28533319]]), 'var_u': 1.0, 'var_v': 1.0, 'w': 0.2}
reward 0
{'u': array([[ 2.07354922,  1.0195585 ,  1.0087719 ],
       [ 0.54179132,  1.19008464,  0.39292773],
       [ 1.0440529 ,  0.46292477,  0.81586203],
       ...,
       [-1.56773349, -0.76497728,  0.22799511],
       [-0.71650132,  1.25277039,  0.55016291],
       [-0.17462832,  1.21656952, -0.3246703 ]]), 'v': array([[-0.99518

ValueError: operands could not be broadcast together with shapes (3,0) (2,) 